In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from pathlib import Path
import pandas as pd
import numpy as np
import itertools
from copy import copy
from pprint import pprint
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import abc
import sys
import re
sys.path.append("../")
import gpusims
import gpusims.plot.metrics as metric
from gpusims.plot.data import PlotData
from gpusims.config import Config, parse_configs
from gpusims.bench import parse_benchmarks

In [5]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [6]:
root_dir = Path("/Users/roman/dev/gpgpusims")
benchmark_dir = root_dir / "benchmarks"
run_dir = root_dir / "run"
assert benchmark_dir.is_dir()
assert run_dir.is_dir()

In [7]:
configs = parse_configs(benchmark_dir / "configs" / "configs.yml")
benchmarks = parse_benchmarks(benchmark_dir / "benchmarks.yml")

pprint(configs)
pprint(benchmarks)
pprint(gpusims.SIMULATORS)

{'sm6_gtx1080': Config(key='sm6_gtx1080', name='GTX 1080', path=PosixPath('/Users/roman/dev/gpgpusims/benchmarks/configs/SM6_GTX1080'), spec={'sm_count': 20, 'clock_speed': 1607}),
 'sm86_rtx3070': Config(key='sm86_rtx3070', name='RTX 3070', path=PosixPath('/Users/roman/dev/gpgpusims/benchmarks/configs/SM86_RTX3070'), spec={'sm_count': 46, 'clock_speed': 1132})}
{'babelstream': Benchmark(/Users/roman/dev/gpgpusims/benchmarks/BabelStream),
 'cuda10-matrixmul': Benchmark(/Users/roman/dev/gpgpusims/benchmarks/CUDA10-matrixMul),
 'cuda10-transpose': Benchmark(/Users/roman/dev/gpgpusims/benchmarks/CUDA10-transpose),
 'cuda4-matrixmul': Benchmark(/Users/roman/dev/gpgpusims/benchmarks/CUDA4-matrixMul),
 'cuda6-transpose': Benchmark(/Users/roman/dev/gpgpusims/benchmarks/CUDA6-transpose),
 'vectoradd': Benchmark(/Users/roman/dev/gpgpusims/benchmarks/vectorAdd)}
{'accelsim-ptx': <class 'gpusims.accelsim.AccelSimPTXBenchmarkConfig'>,
 'accelsim-sass': <class 'gpusims.accelsim_sass.AccelSimSASSBen

In [13]:
selected_simulators = copy(gpusims.SIMULATORS)
selected_configs = copy(configs)
selected_benchmarks = copy(benchmarks)

# for testing
if False:
    testing_simulators = None
    testing_configs = None
    testing_benchmarks = None
    # testing_simulators = list(gpusims.SIMULATORS.keys())
    # testing_simulators = [gpusims.NATIVE]
    # testing_simulators = [gpusims.MULTI2SIM]
    # testing_configs = ["sm6_gtx1080"]
    # testing_benchmarks = ["babelstream"] # "cuda6-transpose"]
    
    if testing_simulators is not None:
        selected_simulators = {k: v for k, v in gpusims.SIMULATORS.items() if k in testing_simulators}
    if testing_configs is not None:
        selected_configs = {k: v for k, v in configs.items() if k in testing_configs}
    if testing_benchmarks is not None:
        selected_benchmarks = {k: v for k, v in benchmarks.items() if k in testing_benchmarks}

pprint(selected_simulators)
pprint(selected_configs)
pprint(selected_benchmarks)

{'accelsim-ptx': <class 'gpusims.accelsim.AccelSimPTXBenchmarkConfig'>,
 'accelsim-sass': <class 'gpusims.accelsim_sass.AccelSimSASSBenchmarkConfig'>,
 'm2s': <class 'gpusims.multi2sim.Multi2SimBenchmarkConfig'>,
 'macsim': <class 'gpusims.macsim.MacSimBenchmarkConfig'>,
 'native': <class 'gpusims.native.NativeBenchmarkConfig'>,
 'tejas': <class 'gpusims.tejas.TejasBenchmarkConfig'>}
{'sm6_gtx1080': Config(key='sm6_gtx1080', name='GTX 1080', path=PosixPath('/Users/roman/dev/gpgpusims/benchmarks/configs/SM6_GTX1080'), spec={'sm_count': 20, 'clock_speed': 1607}),
 'sm86_rtx3070': Config(key='sm86_rtx3070', name='RTX 3070', path=PosixPath('/Users/roman/dev/gpgpusims/benchmarks/configs/SM86_RTX3070'), spec={'sm_count': 46, 'clock_speed': 1132})}
{'babelstream': Benchmark(/Users/roman/dev/gpgpusims/benchmarks/BabelStream),
 'cuda10-matrixmul': Benchmark(/Users/roman/dev/gpgpusims/benchmarks/CUDA10-matrixMul),
 'cuda10-transpose': Benchmark(/Users/roman/dev/gpgpusims/benchmarks/CUDA10-transp

In [14]:
colors = ["#5F34FA", "#49DFE3", "#8CFA5D", "#E3BC49", "#FF7357", "#EE34FA"]
sim_color = {
    "GPUTejas": colors[0],
    "AccelSim PTX": colors[1],
    "AccelSim SASS": colors[2],
    "Hardware": "#FF3C1E", # colors[3],
    "Multi2Sim": colors[4],
    "MacSim": colors[5],
}
abbr = {
    "GPUTejas": "Tejas",
    "AccelSim PTX": "Accel PTX",
    "AccelSim SASS": "Accel SASS",
    "Hardware": "HW",
    "Multi2Sim": "M2S",
    "MacSim": "MS",
}
margin = 50
default_layout_options = dict(
    plot_bgcolor="white",
    margin=dict(
        pad=10,
        autoexpand=True,
        l=margin, r=margin, t=1.5*margin, b=margin
    ),
    width=900,
    height=500,
)

In [15]:
def plot_bars(metric_cls, data, config, title=None, fontsize=14, font_family="Helvetica"):
    bars = []
    data = data.sort_values(by=["Simulator", "Benchmark"])
    data = data.set_index(["Simulator"])
    simulators = data.index.get_level_values("Simulator").unique().tolist()
    benchmarks = data["Benchmark"].unique().tolist()
    
    for i, sim in enumerate(simulators):
        # add hardware GPU name
        name = sim
        if sim == "Hardware":
            name += f" ({config.name})"
        
        sim_data = data.loc[[sim]]
        
        bars.append(go.Bar(
            x = sim_data["Benchmark"],
            y = sim_data["Value"],
            text = sim_data["Value"],
            textposition='auto',
            # textposition='inside',
            textangle=90,
            textfont = dict(
                # color="white",
                size=fontsize - 2,
                color="black",
            ),
            # hovertext=name,
            hovertemplate = (
                "<b>%{x}</b><br>" +
                "%{y:.2f}<br>"
            ),
            name=str(name),
            # marker=go.bar.Marker(color=colors[i % len(colors)]),
            marker=dict(
                color=sim_color[sim],
                line=dict(
                    color='rgba(50, 171, 96, 1.0)',
                    width=0
                ),
            ),
        ))

    yaxis_text = metric_cls.name
    if metric_cls.unit is not None:
        yaxis_text += f" [{metric_cls.unit}]"
    if title is None:
        title = f"{metric_cls.name} ({config.name})"
    # if len(benchmarks) == 1:
    #     title = f"{metric_cls.name} for {benchmarks[0]} ({config.name})"
    layout = go.Layout(
        font_family=font_family,
        font_color="black",
        font_size=fontsize,
        title=dict(
            text=title,
            x=0.5,
            y=0.95,
            xanchor="center",
            yanchor="top",
        ),
        yaxis=go.layout.YAxis(
            title=yaxis_text,
            gridcolor="gray",
            zerolinecolor="gray",
            # type="log",
            # range=[min_axis_val * 0.9 ,max_axis_val*1.1]
        ),
        xaxis=go.layout.XAxis(
            tickfont=dict(
                size=0.8 * fontsize,
            ),
            dividerwidth=0,
            dividercolor="white",
        ),
        hoverlabel=dict(
            bgcolor="white",
            font_size=fontsize,
            font_family=font_family,
        ),
        barmode="group",
        bargroupgap=0.1,
        bargap=0.25,
        showlegend=True,
        **default_layout_options,
    )
    if metric.config.get("log"):
        layout.yaxis.type = "log"
    fig = go.Figure(data=bars, layout=layout)
    return fig

In [16]:
def plot_bars_exec_time(
    data, config, metric_cls,
    title=None, fontsize=14, font_family="Helvetica", round_to=2,
):
    bars = []
    data = data.set_index(["Simulator", "Kind"])
    data = data.sort_values(by="Benchmark")
    data = data.sort_index()
    simulators = data.index.get_level_values("Simulator").unique().tolist()
    
    for i, sim in enumerate(simulators):
        # add hardware GPU name
        name = sim
        if sim == "Hardware":
            name += f" ({config.name})"
        
        # add bars
        trace_time = data.loc[data.index == (sim, "Trace")]
        sim_time = data.loc[data.index == (sim, "Sim")]
        
        benchmarks = trace_time["Benchmark"].values
        x = [benchmarks, [abbr[sim]] * len(benchmarks)]
        # flat_x = [item for sublist in x for item in sublist]
        # y = [10] * len(flat_x)
        y = trace_time["Value"]
        if sim != "Hardware":
            y = y.round(round_to)
        
        bars.append(go.Bar(
            x = x,
            y = y,
            text = y.apply(lambda v: f"tracing<br>{v}"),
            textposition='auto',
            textangle=0,
            showlegend=False,
            textfont = dict(
                size=fontsize - 2,
                color="black",
            ),
            hovertemplate = (
                "<b>%{x}</b><br>" +
                "%{y:.2f}<br>"
            ),
            name=str(name),
            marker=dict(
                color=sim_color[sim],
                line=dict(
                    color="white",
                    width=2,
                ),
            ),
        ))
        
        # continue
        total_time = trace_time["Value"].values +  sim_time["Value"].values
        if sim != "Hardware":
            total_time = total_time.round(round_to)
        # print(total_time)
        bars.append(go.Bar(
            x = x,
            y = total_time,
            text = total_time,
            textposition='auto',
            textangle=0,
            textfont = dict(
                size=fontsize - 2,
                color="black",
            ),
            hovertemplate = (
                "<b>%{x}</b><br>" +
                "%{y:.2f}<br>"
            ),
            name=str(name),
            marker=dict(
                color=sim_color[sim],
                line=dict(
                    color="white",
                    width=2,
                ),
            ),
        ))

    margin = 50
    yaxis_text = metric_cls.name
    if metric_cls.unit is not None:
        yaxis_text += f" [{metric_cls.unit}]"
    if title is None:
        title = f"{metric_cls.name} ({config.name})"
    # if len(benchmarks) == 1:
    #     title = f"{metric_name} for {benchmarks[0]} ({config.name})"
    layout = go.Layout(
        font_family=font_family,
        font_color="black",
        font_size=fontsize,
        title=dict(
            text=title,
            x=0.5,
            y=0.95,
            xanchor="center",
            yanchor="top",
        ),
        yaxis=go.layout.YAxis(
            title=yaxis_text,
            gridcolor="gray",
            zerolinecolor="gray",
            # range=[min_axis_val * 0.9 ,max_axis_val*1.1]
        ),
        xaxis=go.layout.XAxis(
            # title="Benchmark",
            tickfont=dict(
                size=0.8 * fontsize,
            ),
            dividerwidth=0,
            dividercolor="white",
        ),
        hoverlabel=dict(
            bgcolor="white",
            font_size=fontsize,
            font_family=font_family,
        ),
        barmode="stack",
        # bargroupgap=0.1,
        bargap=0.1,
        showlegend=True,
        **default_layout_options,
    )
    if metric.config.get("log"):
        layout.yaxis.type = "log"
    return go.Figure(data=bars, layout=layout)

In [17]:
# => per config, benchmark and input, plot bars for each simulator

metrics = {
    gpusims.plot.metrics.ExecutionTime: plot_bars_exec_time,
    gpusims.plot.metrics.Cycles: plot_bars,
    gpusims.plot.metrics.L2ReadHit: plot_bars,
    gpusims.plot.metrics.DRAMReads: plot_bars,
    gpusims.plot.metrics.DRAMWrites: plot_bars,
    gpusims.plot.metrics.IPC: plot_bars,
    gpusims.plot.metrics.InstructionCount: plot_bars,
}

for metric_cls, metrics_plot_func in metrics.items():
    for (config_name, config), (bench_name, bench) in itertools.product(
        selected_configs.items(),
        selected_benchmarks.items()
    ):
        print(config_name, bench_name)
        all_metric_df = []
        for inp in bench.inputs:
            plot_data = PlotData(benchmark=bench, config=config, inp=inp)
            for (sim_name, sim) in selected_simulators.items():
                if not bench.enabled(sim_name):
                    continue
                # print(sim_name, config_name, bench_name)
                bench_config = sim(
                    run_dir=run_dir / sim_name.lower(),
                    benchmark=bench,
                    config=config,
                )
                if not bench_config.input_path(inp).is_dir():
                    print(f"WARN: {bench_config.input_path(inp)} does not exist")
                    continue

                plot_data[sim_name] = bench_config.load_dataframe(inp)

            metric = metric_cls(plot_data)
            metric_df = metric.compute()
            metric_df["Benchmark"] = f"{bench.name}<br>{inp.args}"

            all_metric_df.append(metric_df)
            # continue

            # print(metric_df)
            fig = metrics_plot_func(
                data=metric_df,
                config=config,
                metric_cls=metric_cls,
                title=f"{metric_cls.name} for {bench.name} {inp.args} ({config.name})"
            )
            filename = ["bar", metric.name, bench.name, config.key, inp.sanitized_name()]
            filename = Path("./figs") / gpusims.utils.slugify("_".join(filename))
            filename = filename.with_suffix(".pdf")
            fig.write_image(filename, format='pdf')
            print("wrote", filename)

        all_metric_df = pd.concat(all_metric_df)
        fig = metrics_plot_func(
            data=all_metric_df,
            config=config,
            metric_cls=metric_cls,
            title=f"{metric_cls.name} for {bench.name} ({config.name})",
        )
        # fig.show()
        filename = ["all_inputs_bar", metric_cls.name, bench.name, config.key]
        filename = Path("./figs") / gpusims.utils.slugify("_".join(filename))
        filename = filename.with_suffix(".pdf")
        fig.write_image(filename, format='pdf')
        print("wrote", filename)
        # break

all_metric_df

sm6_gtx1080 vectoradd
wrote figs/bar_execution-time_vectoradd_sm6_gtx1080_input-1000.pdf
wrote figs/bar_execution-time_vectoradd_sm6_gtx1080_input-1000000.pdf
wrote figs/all_inputs_bar_execution-time_vectoradd_sm6_gtx1080.pdf
sm6_gtx1080 cuda10-transpose
wrote figs/bar_execution-time_cuda10-transpose_sm6_gtx1080_input-repeat1-dimx32-dimy32.pdf
wrote figs/bar_execution-time_cuda10-transpose_sm6_gtx1080_input-repeat1-dimx64-dimy64.pdf
wrote figs/bar_execution-time_cuda10-transpose_sm6_gtx1080_input-repeat3-dimx32-dimy32.pdf
wrote figs/all_inputs_bar_execution-time_cuda10-transpose_sm6_gtx1080.pdf
sm6_gtx1080 cuda6-transpose
wrote figs/bar_execution-time_cuda6-transpose_sm6_gtx1080_input-repeat1-dimx32-dimy32.pdf
wrote figs/bar_execution-time_cuda6-transpose_sm6_gtx1080_input-repeat1-dimx64-dimy64.pdf
wrote figs/bar_execution-time_cuda6-transpose_sm6_gtx1080_input-repeat3-dimx32-dimy32.pdf
wrote figs/all_inputs_bar_execution-time_cuda6-transpose_sm6_gtx1080.pdf
sm6_gtx1080 cuda10-matrixmu

KeyError: "None of ['Simulator'] are in the columns"

In [118]:
# debug dataframe
# sim = gpusims.MULTI2SIM
# sim = gpusims.MACSIM
sim = gpusims.TEJAS
# sim = gpusims.ACCELSIM_SASS
# sim = gpusims.NATIVE
# benchmark = benchmarks["cuda4-matrixmul"]
benchmark = benchmarks["cuda6-transpose"]
# config=configs["sm86_rtx3070"]
config=configs["sm6_gtx1080"]
bench_config = gpusims.SIMULATORS[sim](
    run_dir=run_dir / sim.lower(),
    benchmark=benchmark,
    config=config,
)
assert bench_config.benchmark.enabled(sim)
bench_input = bench_config.benchmark.inputs[0]
df = bench_config.load_dataframe(bench_input)
# pprint(df.columns.tolist())
# df.T[~df.T.index.str.contains(r"SM \d+")]
df # .T

,emulator_type,created,sim_time_secs,total_inst_count,total_cycle_count,dram_total_reads,dram_total_writes,dram_avg_read_latency,total_instr_cache_access,total_const_cache_access,total_shared_cache_access,total_instr_cache_misses,total_const_cache_misses,total_shared_cache_misses,kips,total_ipc,trace_wall_time,sim_wall_time
0,OCELOT,NaN,18.341,2304,9440,0,0,0.0,0,0,0,8,68,0,0.12562,0.244068,0.454692,21.817216
